In [2]:
import datetime
import itertools
import numpy as np
import pandas as pd

import rdflib
import owlrl
import pyshacl

#from rdflib import Graph, Literal, URIRef, BNode
#from rdflib.term import Identifier
#from rdflib.collection import Collection
#from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
#import rdflib.plugins.sparql.update
#import owlrl.RDFSClosure

# Create KG resources from CSV dumps

## Load CSV file

In [3]:
studies_df = pd.read_csv("./data/dataset.csv", sep=';')
studies_df.head()

,title,abstract,topic,tags,Group,Authors,Release Date,Identifier,Permanent Identifier (DOI),Permanent Identifier (URI),...,License,Public Access Level,Contact Name,Contact Email,Plasma Source Procedure,Plasma Medium Procedure,Plasma Diagnostics Procedure,Plasma Target Name,Plasma Target Properties,Plasma Target Procedure
0,Verified modeling of a low pressure hydrogen p...,A self-consistent ﬂuid model has been successf...,Materials / Surfaces; Plasma Chemical Processes,ECR plasma; Model validation; fluid modelling;...,INP,"Sigeneger, Florian; Ellis, James; Harhausen, J...",2022-10-10,dc66dc03-a157-4179-8d8c-19999bcfe8ff,10.34711/inptdat.599,https://www.inptdat.de/node/599,...,Creative Commons Attribution 4.0 International...,Public,Florian Sigeneger,sigeneger@inp-greifswald.de,NaN,NaN,NaN,NaN,NaN,NaN
1,Effect of a spatially fluctuating heating of p...,The work is concerned with the effect of a spa...,Materials / Surfaces,plasma processing,INP,"Zhu, Tao; Baeva, Margarita; Testrich, Holger; ...",2022-05-13,7a581f22-499c-4050-b2bb-09dab8f881bd,10.34711/inptdat.571,https://www.inptdat.de/node/571,...,Creative Commons Attribution 4.0 International...,Public,Margarita Baeva,baeva@inp-greifswald.de,Al2O3 powder is injected into the plasma jet. ...,A plasma jet is generated in a mixture of Ar (...,NaN,NaN,NaN,NaN
2,Upscaling from single- to multi-filament diele...,A study on the scalability of discharge charac...,Plasma Chemical Processes,streamer discharge; basic research,INP,"Höft, Hans; Becker, Markus M.; Kettlitz, Manfr...",2022-08-26,51e4aceb-eb8c-44d4-ac6a-295fdbd79d84,10.34711/inptdat.572,https://www.inptdat.de/node/572,...,Creative Commons Attribution 4.0 International...,Public,"Höft, Hans",hans.hoeft@inp-greifswald.de,Unipolar square wave pulses with 10kV amplitud...,The total gas flow through the cell was set to...,Optical diagnostics were performed with a fast...,NaN,NaN,NaN
3,The effect of oxygen admixture on the properti...,This work presents the datasets of the results...,Plasma Chemical Processes,argon; atmospheric pressure; fluid modelling; ...,INP; Leibniz-IPHT,"Baeva, Margarita; Stankov, Marjan; Trautvetter...",2021-03-01,adeb6746-9957-4b9c-afcf-f9b2e14b21bc,10.34711/inptdat.378,https://www.inptdat.de/node/378,...,Creative Commons Attribution 4.0 International...,Public,"Baeva, Margarita",baeva@inp-greifswald.de,The microwave plasma is generated in a quartz ...,The increase of the O2-admixture has to be fol...,NaN,NaN,NaN,NaN
4,Modelling and experimental evidence of the cat...,The lifetime of tungsten cathodes used in plas...,Materials / Surfaces; Welding / Switching,plasma spray torch; erosion; tungsten cathode;...,INP; Universidade da Madeira,"Baeva, Margarita; Benilov, Mikhail S.; Zhu, Ta...",2022-07-07,f7116dbe-455b-451d-b7d7-8a8a48d82993,10.34711/inptdat.512,https://www.inptdat.de/node/512,...,Creative Commons Attribution 4.0 International...,Public,Margarita Baeva,baeva@inp-greifswald.de,NaN,NaN,The erosion of the cathode made of lanthanated...,tungsten,Tungsten can withstand temperatures up to abou...,"Due to the erosion, the cathode has to be repl..."


In [4]:
studies_df.describe()

,title,abstract,topic,tags,Group,Authors,Release Date,Identifier,Permanent Identifier (DOI),Permanent Identifier (URI),...,License,Public Access Level,Contact Name,Contact Email,Plasma Source Procedure,Plasma Medium Procedure,Plasma Diagnostics Procedure,Plasma Target Name,Plasma Target Properties,Plasma Target Procedure
count,29,29,29,27,28,29,29,29,29,29,...,29,29,29,29,14,7,18,6,6,6
unique,29,29,6,26,12,26,26,29,29,29,...,1,1,16,14,12,6,18,5,6,5
top,Verified modeling of a low pressure hydrogen p...,A self-consistent ﬂuid model has been successf...,Plasma Chemical Processes,GMAW; high-current arc,INP,"Loffhagen, Detlef; Pinhao, Nuno R.; Vass, Mate...",2020-01-30,dc66dc03-a157-4179-8d8c-19999bcfe8ff,10.34711/inptdat.599,https://www.inptdat.de/node/599,...,Creative Commons Attribution 4.0 International...,Public,"Baeva, Margarita",baeva@inp-greifswald.de,The drift cell is situated within a vacuum cha...,Standard conditions of the argon gas are assured.,Optical diagnostics were performed with a fast...,mild steel,Tungsten can withstand temperatures up to abou...,Bead-on-plate welding
freq,1,1,9,2,15,3,3,1,1,1,...,29,29,4,7,3,2,1,2,1,2


### Preprocess CSV

In [75]:
studies_df["formatted_authors"] = [[tuple([p.strip() for p in n.split(',')]) for n in row.split(';')] 
                           if row==row else [] for row in studies_df["Authors"]]
studies_df["formatted_topic"] = [[p.strip() for p in row.split(';')] if row==row else [] for row in studies_df["topic"]]
studies_df["formatted_tags"] = [[p.strip() for p in row.split(';')] if row==row else [] for row in studies_df["tags"]]
studies_df["formatted_Source_Name"] = [[p.strip() for p in row.split(';')] if row==row else [] for row in studies_df["Plasma Source Name"]]
studies_df["formatted_Target_Name"] = [[p.strip() for p in row.split(';')] if row==row else [] for row in studies_df["Plasma Target Name"]]
studies_df["formatted_Medium_Name"] = [[p.strip() for p in row.split(';')] if row==row else [] for row in studies_df["Plasma Medium Name"]]

## Create KG
KG
Controller

In [6]:
class KG_Controller():    
    def __init__(self, namespace, prefix, index):
        self.namespace=namespace
        self.prefix=prefix
        self.index=index

    def getNewEntity(self,):
        entity = rdflib.URIRef(self.namespace + self.prefix + f"{self.index}")
        self.index +=1 
        return entity

    def getNewEntities(self, length):
        return [self.getNewEntity() for _ in range(length)]

# TODO get init value from SPARQL query, add query to check if resource already exists 
controller = KG_Controller(rdflib.Namespace("http://qptdat.plasma-mds.org/resource/"), "e", 10)
g = rdflib.Graph()

In [7]:
rdf_namespace = rdflib.Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns")
rdfs_namespace = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")
owl_namespace = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
xsd_namespace = rdflib.Namespace("http://www.w3.org/2001/XMLSchema#")
foaf_namespace = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
qpt_namespace = rdflib.Namespace("http://qptdat.plasma-mds.org/ontology/")
qpt_source_namespace = rdflib.Namespace("http://qptdat.plasma-mds.org/source_ontology/")
qptkg_namespace = rdflib.Namespace("http://qptdat.plasma-mds.org/resource/")

In [8]:
remote_graph = rdflib.Graph()
remote_graph.bind("qpt", qpt_namespace)
remote_graph.bind("qpt_source", qpt_source_namespace)
remote_graph.bind("rdf", rdf_namespace)
remote_graph.bind("rdfs", rdfs_namespace)
remote_graph.bind("owl", owl_namespace)
remote_graph.bind("xsd", xsd_namespace)
remote_graph.bind("foaf", foaf_namespace)
remote_graph.bind("qptkg", qptkg_namespace)

### Create plasma studies 

In [9]:
studies_dict = {i:controller.getNewEntity() for i in studies_df.index}

In [10]:
for k,v in studies_dict.items():
    g.add((v,rdf_namespace.type,qpt_namespace.PlasmaStudy))
    
    g.add((v,qpt_namespace.studyTitle,rdflib.Literal(studies_df["title"].loc[k], lang="en")))
    g.add((v,qpt_namespace.studyAbstract,rdflib.Literal(studies_df["abstract"].loc[k], lang="en")))   

### Create dataset

In [11]:
dataset_dict = {i:controller.getNewEntity() for i in studies_df.index}
for k,v in dataset_dict.items():
    g.add((v, rdf_namespace.type, qpt_namespace.Dataset))
    g.add((studies_dict[k], qpt_namespace.hasDatasetOutput, dataset_dict[k]))
    
    dataset_title = studies_df["title"].loc[k]
    g.add((v, rdfs_namespace.label, rdflib.Literal(dataset_title, lang="en")))
    if studies_df["Release Date"].loc[k]:
        publication_date = datetime.date.fromisoformat(studies_df["Release Date"].loc[k])
        g.add((v, qpt_namespace.publicationDate, rdflib.Literal(publication_date, datatype=xsd_namespace.date) ))
    if studies_df['Permanent Identifier (DOI)'].loc[k]:
        doi = f"http://doi.org/{studies_df['Permanent Identifier (DOI)'].loc[k]}"
        g.add((v, qpt_namespace.doi, rdflib.URIRef(doi)))
    if studies_df["Is supplementing"].loc[k]:
        supplementing = f"http://doi.org/{studies_df['Is supplementing'].loc[k]}"
        g.add((v, qpt_namespace.isSupplementing, rdflib.URIRef(supplementing))) 
        g.add((studies_dict[k], qpt_namespace.isSupplementing, rdflib.URIRef(supplementing)))

### topics

In [12]:
search_dict = {f"\"^{x}$\"":x for x in set(itertools.chain(*studies_df["formatted_topic"])) }

In [13]:
topic_query = """
SELECT ?search ?topic
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?topic a qpt:ResearchTopic .
        ?topic rdfs:label ?label .
        FILTER (LANG(?label) = 'en') 
        FILTER( REGEX(?label, ?search, "i") )

        VALUES ?search { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""
qres = remote_graph.query(topic_query)
result_dict = {search_dict[f"\"{row.search}\""]:row.topic for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

unmapped strings: ['Plasma Medicine', 'Plasma Chemical Processes', 'Welding / Switching', 'Materials / Surfaces']


In [14]:
for i,row in studies_df.iterrows():
    for topic in row["formatted_topic"]: 
        if topic in result_dict:
            g.add((studies_dict[i] ,qpt_namespace.hasResearchTopic, result_dict[topic] ))

### tags

In [15]:
search_dict = {f"\"^{x}$\"":x for x in set(itertools.chain(*studies_df["formatted_tags"])) }

In [16]:
tag_query = """
SELECT ?search ?topic
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?topic a qpt:PlasmaStudyTag .
        ?topic rdfs:label ?label .
        FILTER (LANG(?label) = 'en') 
        FILTER( REGEX(?label, ?search, "i") )

        VALUES ?search { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""
qres = remote_graph.query(tag_query)
result_dict = {search_dict[f"\"{row.search}\""]:row.topic for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

unmapped strings: ['basic research', 'heat-sensitive surfaces', 'copper electrodes', 'XPS', 'electron swarm parameters', 'Ammonia production mechanisms', 'GMAW', 'benchmark data', 'infrared laser', 'absorption spectroscopy', 'electric arcs', 'EPR spectroscopy', 'experiment', 'ECR plasma', 'argon', 'electron Boltzmann equation', 'thermal diagnostic', 'EEDF', 'evaporation', 'tungsten cathode', 'TALIF', 'self-controlling mechanism', 'field reversal', 'microarc', 'plasma jet', 'DBD', 'plasma spraying', 'unified description', 'atmospheric pressure', 'plasma chemistry', 'reactive oxygen species', 'plasma polymerization', 'HMDSO', 'plasma modelling/simulation', 'MHD model', 'ultra low-k etching', 'plasma spray torch', 'microwave plasma', 'C2H2', 'space charge', 'C2H6', 'self-organisation', 'Monte Carlo Flux', 'convection', 'radicals', 'ionization layer', 'streamer discharge', 'Model validation', 'plasma parameters', 'erosion', 'TMS', 'high-current arc', 'fluid modelling', 'DC torch', 'C2H4', 

In [17]:
for i,row in studies_df.iterrows():
    for tag in row["formatted_tags"]: 
        if tag in result_dict:
            g.add((studies_dict[i] ,qpt_namespace.hasPlasmaStudyTag, result_dict[tag] ))

### Authors


In [61]:
search_dict = {f"(\"{x[0]}\"@en \"{x[1]}\"@en)":x for x in set(itertools.chain(*studies_df["formatted_authors"])) }

In [19]:
person_query = """
SELECT ?search_surname ?search_givenname ?person
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?person a foaf:Person .
        ?person foaf:surname ?search_surname .
        ?person foaf:givenname ?search_givenname .

        VALUES (?search_surname ?search_givenname)  { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""# TODO change to qpt:Person
qres = remote_graph.query(person_query)
result_dict = {search_dict[f"(\"{row.search_surname}\"@en \"{row.search_givenname}\"@en)"]:
               row.person for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

unmapped strings: [('Veklich', 'Anatoly'), ('Zhang', 'Guokai'), ('Longo', 'Savino'), ('Testrich', 'Holger'), ('Korolov', 'Ihor'), ('Lozano', 'Philipp'), ('Benilov', 'Mikhail S.'), ('Niemi', 'Kari'), ('Schäfer', 'Jan'), ('Zimmermann', 'Sven'), ('Manfred', 'Katherine M.'), ('Weltmann', 'Klaus-Dieter'), ('Gorbanev', 'Yury'), ('Kählert', 'Hanno'), ('Ellis', 'James'), ('Köpp', 'Daniel'), ('Lang', 'Norbert'), ('Gonzalez', 'Diego'), ('Semenov', 'Igor'), ('Schulz', 'Stefan'), ('Hempel', 'Frank'), ('Kewitz', 'Thorben'), ('Bosnjakovic', 'Danko'), ('Stankov', 'Marjan'), ('Dujko', 'Sasa'), ('Czerny', 'Andreas K.'), ('Pinhao', 'Nuno R.'), ('Klages', 'Claus-Peter'), ('Loffhagen', 'Detlef'), ('Harhausen', 'Jens'), ('Gött', 'Gregor'), ("O'Connell", 'Deborah'), ('Wubs', 'Jente'), ('Diomede', 'Paola'), ('Norman', 'Helen C.'), ('Becker', 'Markus M.'), ('Mohsni', 'Chayma'), ('Sharma', 'Rahul'), ('Brandenburg', 'Ronny'), ('Sun', 'Anbang'), ('van Helden', 'Jean-Pierre'), ('Charrada', 'Kamel'), ('Uhrlandt', 

Add new persons to KG

In [20]:
person_dict = {p:controller.getNewEntity() for p in unmapped}
for k,v in person_dict.items():
    g.add((v, rdf_namespace.type, foaf_namespace.Person))
    g.add((v, foaf_namespace.givenname , rdflib.Literal(k[1], lang="en")))
    g.add((v, foaf_namespace.surename , rdflib.Literal(k[0], lang="en")))
    g.add((v, foaf_namespace.name , rdflib.Literal(f"{k[1]} {k[0]}", lang="en")))
    g.add((v, rdf_namespace.label , rdflib.Literal(f"{k[1]} {k[0]}", lang="en")))
    result_dict[k]=v

In [21]:
for i,row in studies_df.iterrows():
    for person in row["formatted_authors"]: 
        if person in result_dict:
            g.add((studies_dict[i], qpt_namespace.contributor, result_dict[person]))
            g.add((dataset_dict[i], qpt_namespace.author, result_dict[person]))

### License, Access Level

In [22]:
license_dict = {"Creative Commons Attribution 4.0 International (CC BY 4.0)":rdflib.URIRef("")}
access_level_dict = {"Public":qpt_namespace.public}

In [23]:
for i,row in studies_df.iterrows():
    if row["License"] in license_dict:
        g.add((dataset_dict[i],qpt_namespace.license, license_dict[row["License"]]))
    if row["Public Access Level"] in access_level_dict:
        g.add((dataset_dict[i],qpt_namespace.access_level, access_level_dict[row["Public Access Level"]]))

### Source
In general we assume each study was performed on *one* specific source. This assumption might not be valid for all plasma studies, but the CSV data does not contain such information. 

We assume the studies ["278af539-7a4e-4fe5-8064-37c53a0ee7f5","8c6093fc-0672-4f1e-8dad-eb414320bead", 
 "95b7984b-bd8d-48f0-b5cf-c469d9fd17ff"] used the same souce base on the avalible data. 

In [24]:
studies_df[["Plasma Source Name","Plasma Source Properties", "Plasma Source Procedure"]].dropna(how="all")

,Plasma Source Name,Plasma Source Properties,Plasma Source Procedure
0,AURA-WAVE (Sairem),Electron cyclotron resonance (ECR) coaxial pla...,NaN
1,Oerlikon Metco F4MB-XL,The plasma spray torch considered in the prese...,Al2O3 powder is injected into the plasma jet. ...
2,DBD; single-filament DBD; multi-filament DBD,Two different arrangements were used. The sing...,Unipolar square wave pulses with 10kV amplitud...
3,atmospheric pressure MW discharge; plasma torch,"For an incoming power of 1 kW, the predicted a...",The microwave plasma is generated in a quartz ...
4,Oerlikon Metco F4MB-XL,"The torch is typically operated in pure argon,...",NaN
5,scanning drift tube apparatus,Time-of-flight (TOF) experiment; ultraviolet l...,The drift cell is situated within a vacuum cha...
6,scanning drift tube apparatus,Time-of-flight (TOF) experiment; ultraviolet l...,The drift cell is situated within a vacuum cha...
7,scanning drift tube apparatus,Time-of-flight (TOF) experiment; ultraviolet l...,The drift cell is situated within a vacuum cha...
8,DBD,Atmospheric-pressure DBD between plane steel m...,NaN
9,single-filament DBD,The object of interest is streamer-surface int...,NaN


In [25]:
source_groups = [["278af539-7a4e-4fe5-8064-37c53a0ee7f5","8c6093fc-0672-4f1e-8dad-eb414320bead", 
 "95b7984b-bd8d-48f0-b5cf-c469d9fd17ff"]]

source_dict=dict()
for group in source_groups:
    group_entity = controller.getNewEntity() 
    for x in group:
        i = studies_df.index[studies_df["Identifier"] == x][0]
        source_dict[i]=group_entity
for i in studies_df.dropna(subset=["Plasma Source Name"]).index:
    if not i in source_dict:
        source_dict[i]=controller.getNewEntity()     

In [39]:
for k,v in source_dict.items():
    g.add((v, rdf_namespace.type, qpt_namespace.Source))
    g.add((studies_dict[k], qpt_namespace.hasSource, v))

#### Find suitable subclasses of source based on the name

In [40]:
search_dict = {f"\"{x}\"":x for x in set(itertools.chain(*studies_df["formatted_Source_Name"])) }

In [41]:
source_query = """
SELECT ?search_label ?source_class
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?source_class rdfs:subClassOf* qpt:Source .
        ?source_class rdfs:label ?source_label .
        FILTER(REGEX(?source_label,?search_label,"i"))

        VALUES ?search_label { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""
qres = remote_graph.query(source_query)
result_dict = {search_dict[f"\"{row.search_label}\""]:
               row.source_class for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

unmapped strings: ['atmospheric pressure MW discharge', 'multi-filament DBD', 'KINPen-sci', 'mircoarc-plasma source', 'ntAPPJ', 'HF plasma jet', 'kHz plasma jet', 'AURA-WAVE (Sairem)', 'kINPen-sci', 'Applied Materials Centura 5200', 'Oerlikon Metco F4MB-XL', 'single-filament DBD', 'DC arc', 'CCP']


In [45]:
for i,row in studies_df.iterrows():
    for source_label in row["formatted_Source_Name"]:
        if source_label in result_dict:
            g.add((source_dict[i],rdf_namespace.type,result_dict[source_label]))

The remaining unstructured data of each source has to be extended manually. Below an example is shown. 

### Target

In [23]:
studies_df[["Plasma Target Name","Plasma Target Properties", "Plasma Target Procedure"]].dropna(how="all")

,Plasma Target Name,Plasma Target Properties,Plasma Target Procedure
4,tungsten,Tungsten can withstand temperatures up to abou...,"Due to the erosion, the cathode has to be repl..."
17,mild steel,"Mild steel: wire material G3Si1, base material...",Bead-on-plate welding
21,mild steel,"Mild steel: wire material ER70 S-6, base mater...",Bead-on-plate welding
23,Si wafer; SiOCH,Cutted pieces of 10 times 10 cm from a 300 mm ...,Each peace is placed on an Al wafer with 200 m...
26,H2O2; H2SO4; NaN3; D2O; PBN; TEMP; TEMPO; sodi...,"Hydrogen peroxide H₂O₂ (30%), sulphuric acid H...","In spin trapping experiments, a 100 mM solutio..."
28,Polyamide 12,Polyamide 12 powder with mean diameter of 60 µ...,Dosis: 10 mg in 50 ms introduced in the argon ...


Each element of the semicolon spearated list is treated as a separete target of a study. 

In [58]:
target_dict = {i:{k:controller.getNewEntity() for k in row} 
               for i,row in studies_df["formatted_Target_Name"].items() if len(row)>0}

{4: {'tungsten': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e235')},
 17: {'mild steel': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e236')},
 21: {'mild steel': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e237')},
 23: {'Si wafer': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e238'),
  'SiOCH': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e239')},
 26: {'H2O2': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e240'),
  'H2SO4': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e241'),
  'NaN3': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e242'),
  'D2O': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e243'),
  'PBN': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e244'),
  'TEMP': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e245'),
  'TEMPO': rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e246'),
  'sodium tosylate': rdflib.term.URIRef('htt

In [60]:
for k,v in target_dict.items():
    for x in v.values():
        g.add((x, rdf_namespace.type, qpt_namespace.Target))
        g.add((studies_dict[k], qpt_namespace.hasTarget, x))

search for chemical compounds to connect to target 

In [63]:
search_dict = {f"\"{x}\"":x for x in set(itertools.chain(*studies_df["formatted_Target_Name"])) }

In [66]:
target_query = """
SELECT ?search_label ?entity
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?entity rdf:type/rdfs:subClassOf* qpt:ChemicalCompound .
        ?entity rdfs:label ?entity_label .
        FILTER(REGEX(?entity_label,?search_label,"i"))

        VALUES ?search_label { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""# TODO check result_dict
qres = remote_graph.query(target_query)
result_dict = {search_dict[f"\"{row.search_label}\""]:
               row.entity for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

unmapped strings: ['tungsten', 'mild steel', 'H2O2', 'NaN3', 'sodium tosylate', 'SiOCH', 'TEMPO', 'TEMP', 'H2O', 'H2SO4', 'Si wafer', 'D2O', 'PBN', 'DMPO', 'potassium bis(oxalato)oxotitanate(IV) dihydrate', 'Polyamide 12', 'DEPMPO']


In [72]:
for i,row in studies_df.iterrows():
    for target in row["formatted_Target_Name"]: 
        if target in result_dict:
            g.add((target_dict[i][target], qpt_namespace.composedOf, result_dict[target]))

(rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e235'), rdflib.term.URIRef('http://qptdat.plasma-mds.org/ontology/composedOf'), 'test')
(rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e236'), rdflib.term.URIRef('http://qptdat.plasma-mds.org/ontology/composedOf'), 'test1')
(rdflib.term.URIRef('http://qptdat.plasma-mds.org/resource/e237'), rdflib.term.URIRef('http://qptdat.plasma-mds.org/ontology/composedOf'), 'test1')


The remaining unstructured data of each target has to be extended manually. Below an example is shown. 

### Medium

In [29]:
studies_df[["Plasma Medium Name","Plasma Medium Properties", "Plasma Medium Procedure"]].dropna(how="all")

,Plasma Medium Name,Plasma Medium Properties,Plasma Medium Procedure
0,H2,Hydrogen with a gas purity of 6.0 and a flow o...,NaN
1,Ar; H2,The plasma is generated in a mixture of Ar (40...,A plasma jet is generated in a mixture of Ar (...
2,N2/O2,A gas mixture of 0.1 vol% O2 (purity 4.8) in N...,The total gas flow through the cell was set to...
3,Ar; Ar/O2,"Mixture of Ar and O2, the O2-admixture is vari...",The increase of the O2-admixture has to be fol...
4,Ar,"Thermal plasma in the arc column, non-equilibr...",NaN
5,C2H6,Gas temperature: 293.15 K\nPressure: 5 to 1000 Pa,NaN
6,NaN,Gas temperature: 293.15 K\nPressure: 5 to 1000 Pa,NaN
7,C2H2,Gas temperature: 293.15 K\nPressure: 5 to 1000 Pa,NaN
8,Ar; TMS,Gas temperature: 300 K; Pressure: 1 atm; Gas m...,NaN
9,Ar,"Pure argon, gas pressure is 760 Torr, constant...",NaN


The semicolon separeted list of names is treated as mixture and each study is connected to only one medium. 

In [87]:
medium_dict = {i:controller.getNewEntity() for i in studies_df.dropna(subset=["Plasma Medium Name", "Plasma Medium Properties", 
                                                                              "Plasma Medium Procedure"], how="all").index}

In [ ]:
search_dict = {f"\"{x}\"":x for x in set(itertools.chain(*studies_df["formatted_Medium_Name"])) }

In [ ]:
medium_query = """
SELECT ?search_label ?entity
WHERE {
    SERVICE <http://qptdat.plasma-mds.org/sparql> {
        ?entity rdf:type/rdfs:subClassOf* qpt:ChemicalCompound .
        ?entity rdfs:label ?entity_label .
        FILTER(REGEX(?entity_label,?search_label,"i"))

        VALUES ?search_label { """ + " ".join(search_dict.keys()) + """ }
    }
}
"""# TODO check result_dict
qres = remote_graph.query(medium_query)
result_dict = {search_dict[f"\"{row.search_label}\""]:
               row.entity for row in qres} 
unmapped = [v for k,v in search_dict.items() if v not in result_dict]
print(f"unmapped strings: {unmapped}")

In [ ]:
for i,row in studies_df.iterrows():
    if i in medium_dict:
        print((medium_dict[i],rdf_namespace.type,qpt_namespace.Medium))
        print((studies_dict[i],qpt_namespace.hasMedium , medium_dict[i]))
        for medium in row["formatted_Medium_Name"]:
            print((medium_dict[i], qpt_namespace.composedOf, result_dict[medium]))

## Validate local experiments with SHACL

In [ ]:
conforms, results_graph, results_text = pyshacl.validate(g,shacl_graph=sg, ont_graph=og, inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)

Write RDF data into local file 

In [ ]:
for s, p, o in g:
    print(f"{s} {p} {o} .")

## Upload results 